# 1. PyTorch Installation

_NOTE_: If you run this notebook on Google Colab or any similar services, there's a possibility that they might have `pytorch` package installed. But if it's on a local machine, run the following cells (depends on what type of machine you have)

## 1.1. CPU

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu


## 1.2. GPU (an alternative to 1.1, if you have a high-end NVIDIA GPU with support of CUDA)

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

# 2. Checking for models and downloading them

_NOTE_: If you have downloaded the models from our [repository], then just create a folder called `models` and put your model files inside it. Otherwise, run the following cells

In [8]:
import os

if 'models' in os.listdir():
    if os.listdir('models'):
        print('Models are there, where are you?')
    else:
        !cd models && wget https://persianocr.cam/models/letters.pt
        !cd models && wget https://persianocr.cam/models/numbers.pt
else:
    os.mkdir('models')
    !cd models && wget https://persianocr.cam/models/letters.pt
    !cd models && wget https://persianocr.cam/models/numbers.pt

Models are there, where are you?
